In [147]:
import math
from sympy import *
from sympy.physics.mechanics import ReferenceFrame
init_printing(use_latex='mathjax')


In [148]:
## Define symbols
x, y, z = symbols('mu gamma psi')

In [149]:
## Elementary rotation matrices Functions:

def C1(angle):
    x = symbols('x')
    Rx = Matrix([
        [1, 0, 0],
        [0, cos(x), sin(x)],
        [0, -sin(x), cos(x)]])
    return Rx.subs(x, angle)


def C2(angle):
    y = symbols('y')
    Ry = Matrix([
        [cos(y), 0, -sin(y)],
        [0,  1, 0],
        [sin(y), 0, cos(y)]])
    return Ry.subs(y, angle)


def C3(angle):
    z = symbols('z')
    Rz = Matrix([
        [cos(z), sin(z), 0],
        [-sin(z),  cos(z), 0],
        [0,    0, 1]])
    return Rz.subs(z, angle)


In [150]:
class IJKReferenceFrame(ReferenceFrame):
    def __init__(self, name):
        super().__init__(name, latexs=['\mathbf{%s}_{%s}' % (
            idx, name) for idx in ("i", "j", "k")])
        self.i = self.x
        self.j = self.y
        self.k = self.z

## 3-2-3 Euler angles rotation matrices
Find the Euler rotation matrix C21 in terms of 3-2-3 Euler angles rotation sequence, with angles ϴ1, ϴ2, ϴ3. Specifically, frame 2 is obtained from frame 1 by:

 - A rotation ϴ1 about the z-axis (3-axis) of frame 1,
 - a rotation ϴ2 about the y-axis (2-axis) of intermediate frame,
 - a rotation ϴ3 about the z-axis (3-axis) of the transformed frame.

In [151]:
# 3-2-3 Euler angles rotation matrices

# Ctot(x,y,z) = C3(x) * C2(y) * C3(z)

C3_x = C3(x)
C2_y = C2(y)
C3_z = C3(z)

R_zyz = C3_z * C2_y * C3_x
R_zyz

⎡-sin(μ)⋅sin(ψ) + cos(γ)⋅cos(μ)⋅cos(ψ)  sin(μ)⋅cos(γ)⋅cos(ψ) + sin(ψ)⋅cos(μ)  
⎢                                                                             
⎢-sin(μ)⋅cos(ψ) - sin(ψ)⋅cos(γ)⋅cos(μ)  -sin(μ)⋅sin(ψ)⋅cos(γ) + cos(μ)⋅cos(ψ) 
⎢                                                                             
⎣            sin(γ)⋅cos(μ)                          sin(γ)⋅sin(μ)             

 -sin(γ)⋅cos(ψ)⎤
               ⎥
 sin(γ)⋅sin(ψ) ⎥
               ⎥
     cos(γ)    ⎦

### Find from the 3-2-3 Euler rotation matrix the appropriate Euler angles

You can get this by looking at the matrix:
1. Since $R_{zyz}$[2,2] = cos($\gamma$) -> $\gamma$ = $cos^{-1}$($R_{zyz}$[2,2])
2. Then you can get $\mu$ by doing $\frac{R_{zyz}[2,1]}{R_{zyz}[2,0]}$. Therefore, you get $tan(\mu)$ = $\frac{R_{zyz}[2,1]}{R_{zyz}[2,0]}$
3. Similarly, you can get $\psi$ by doing $\frac{R_{zyz}[1,2]}{R_{zyz}[0,2]}$

Note that the notation used for indices corresponds to that of Python

In [152]:
x = trigsimp(atan(R_zyz[2, 1] / R_zyz[2, 0]))
y = trigsimp(acos(R_zyz[2, 2]))
z = -trigsimp(atan(R_zyz[1, 2] / R_zyz[0, 2]))
x,y,z

(atan(tan(μ)), acos(cos(γ)), atan(tan(ψ)))

### For the 3-2-3 Euler sequence, derive the kinematic differential equation

In [249]:
x, y, z = symbols('mu gamma psi')

# C3_x = C3(x)
# C2_y = C2(y)
# C3_z = C3(z)

dotx, doty, dotz = symbols('\dot{\mu} \dot{\gamma} \dot{\psi}')
omegax, omegay, omegaz = symbols('\omega_mu \omega_gamma \omega_psi')

matrix = (Matrix([0, 0, dotz])) + C3_z*(Matrix([0, doty, 0])) + \
    C3_z * C2_y * (Matrix([0, 0, dotz]))  # REVIEW
matrix


⎡\dot{\gamma}⋅sin(ψ) - \dot{\psi}⋅sin(γ)⋅cos(ψ)⎤
⎢                                              ⎥
⎢\dot{\gamma}⋅cos(ψ) + \dot{\psi}⋅sin(γ)⋅sin(ψ)⎥
⎢                                              ⎥
⎣        \dot{\psi}⋅cos(γ) + \dot{\psi}        ⎦

In [237]:
matrix.T * Matrix([dotx, doty, dotz])


[\dot{\gamma}⋅(\dot{\gamma}⋅cos(ψ) + \dot{\psi}⋅sin(γ)⋅sin(ψ)) + \dot{\mu}⋅(\d
ot{\gamma}⋅sin(ψ) - \dot{\psi}⋅sin(γ)⋅cos(ψ)) + \dot{\psi}⋅(\dot{\psi}⋅cos(γ) 
+ \dot{\psi})]

In [258]:
def reverse_matrix_multiply(matrix): # NEED TO CHECK HOW TO TAKE ONLY PART OF COMPONENT THAT GOES WITH EACH THING
    return Matrix([
        [ matrix[0], matrix[0], matrix[0]],
        [ matrix[1], matrix[1], matrix[1]],
        [ matrix[2], matrix[2], matrix[2]]
        ])
# PODRÍA PLANTEAR COMO ECUACIÓN, NO? i.e. [3x3 matrix] * [3x1 vector] = [3x1 vector],
#  knowns would be [dotx, doty, dotz] and matrix (the 3x1 vector)

#def reverse_matrix_multiply(matrix):
    #""" matrix input has 3x1 shape, Matrix33 output has 3x3 shape """
    #Matrix33 = []
    #return Matrix33 * Matrix([[dotx], [doty], [dotz]]) == matrix
#    return [a*xx + b*yy + czz - matrix[0], d*xx + e*yy, + f*zz - matrix[1],
#            g*xx + h*yy + i*zz - matrix[2]]


#dot_angles = Matrix([dotx, doty, dotz])
#reverse_matrix_multiply(matrix)
#matrix.dot(Matrix([dotx, doty, dotz]))
#matrix/dot_angles
#dot_angles*matrix**-1
#from sympy import 
#xx, yy, zz = symbols('x y z')
a, b, c, d, e, f, g, h, i = symbols('a b c d e f g h i')
M = Matrix([[a*dotx + b*doty + c*dotz], [d*dotx + e*doty + f*dotz], [g*dotx + h*doty + i*dotz] ])
M

#solve_linear_system(M, dotx, doty, dotz)


⎡\dot{\gamma}⋅b + \dot{\mu}⋅a + \dot{\psi}⋅c⎤
⎢                                           ⎥
⎢\dot{\gamma}⋅e + \dot{\mu}⋅d + \dot{\psi}⋅f⎥
⎢                                           ⎥
⎣\dot{\gamma}⋅h + \dot{\mu}⋅g + \dot{\psi}⋅i⎦

In [247]:
matrix = Matrix([[-cos(z)*sin(y), sin(z), 0],
                [sin(z)*sin(y), cos(z), 0],
                [cos(y), 0, 1]]) # If i could make this automatically it would be better


matrix, Matrix([dotx, doty, dotz])


⎛⎡-sin(γ)⋅cos(ψ)  sin(ψ)  0⎤  ⎡ \dot{\mu}  ⎤⎞
⎜⎢                         ⎥  ⎢            ⎥⎟
⎜⎢sin(γ)⋅sin(ψ)   cos(ψ)  0⎥, ⎢\dot{\gamma}⎥⎟
⎜⎢                         ⎥  ⎢            ⎥⎟
⎝⎣    cos(γ)        0     1⎦  ⎣ \dot{\psi} ⎦⎠

In [200]:
matrix * Matrix([dotx, doty, dotz]) # CHECKING IF THIS HOLDS UP


⎡\dot{\gamma}⋅sin(ψ) - \dot{\mu}⋅sin(γ)⋅cos(ψ)⎤
⎢                                             ⎥
⎢\dot{\gamma}⋅cos(ψ) + \dot{\mu}⋅sin(γ)⋅sin(ψ)⎥
⎢                                             ⎥
⎣        \dot{\mu}⋅cos(γ) + \dot{\psi}        ⎦

To check if the inversion matrix is the one given in the problem, we compute the following:

In [173]:
invmatrix =simplify(matrix.inv())  
invmatrix

⎡-cos(ψ)    sin(ψ)    ⎤
⎢────────   ──────   0⎥
⎢ sin(γ)    sin(γ)    ⎥
⎢                     ⎥
⎢ sin(ψ)    cos(ψ)   0⎥
⎢                     ⎥
⎢ cos(ψ)   -sin(ψ)    ⎥
⎢ ──────   ────────  1⎥
⎣ tan(γ)    tan(γ)    ⎦

In [180]:
# Show how the matrix would look like (instead of a comma it should be a dot product)
1/sin(y), simplify(invmatrix*sin(y))


⎛        ⎡   -cos(ψ)         sin(ψ)        0   ⎤⎞
⎜  1     ⎢                                     ⎥⎟
⎜──────, ⎢sin(γ)⋅sin(ψ)  sin(γ)⋅cos(ψ)     0   ⎥⎟
⎜sin(γ)  ⎢                                     ⎥⎟
⎝        ⎣cos(γ)⋅cos(ψ)  -sin(ψ)⋅cos(γ)  sin(γ)⎦⎠

In [190]:
# Check if correct by multiplying the original matrix by its inverse. 
# Should get an Identity matrix

simplify(matrix*invmatrix)

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [192]:
if eye(3) == simplify(matrix*invmatrix):
    print("The matrix is correct")
else:
    print("The matrix is incorrect")

The matrix is correct
